## Initialization
- Import the package
- Import the data

In [1]:
import csv
import numpy as np
import pandas as pd
from collections import defaultdict
import string
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import linear_model
from sklearn import metrics # confusion matrix, MSE etc.

In [2]:
path_to_data = '/Users/t.z.cheng/Google_Drive/Research/Delaydoesmatter/real_exp/exp4_20CR12/results_shortlongdelay_2021/v2_20CR12_clean_n67.csv'

In [3]:
## Read the data
df = pd.read_csv(path_to_data)
# df_clean = df
## Take a look of the dataset
# df.head()
# df.tail()
# df.loc[558:663] # see specific rows of data

In [4]:
## Clean up task and subject ID (first five characters)
df['task'] = df['trial_template'].apply(lambda x: x.split("_")[0])
df['sub_id'] = df['participant_id'].apply(lambda x: x.split()[0][0:5])

In [5]:
## Add the accuracy and PPS column to the dataset: transform True and Shorter to 1, False and longer to 0
Correct = []
Shorter = []
for i in np.arange(0,len(df)):
    if df['response_correct'][i] == True:
        Correct.append(1)
    else: 
        Correct.append(0)
    if df['response_value'][i] == "Shorter":
        Shorter.append(1)
    else: 
        Shorter.append(0)
df['Correct'] = Correct
df['Shorter'] = Shorter

In [6]:
## for exp8abc
df['exp'] = df['group_id'].apply(lambda x: x.split("_")[0])
df_exp8b = df[(df['exp'] == 'EXP8b')].reset_index(drop = True)
df = df_exp8b

## Set parameters
- Accuracy threshold for the easiest trials of the main task 
- Extreme RT threshold
- Catch trial accuracy

In [10]:
## Parameters
threshold = .55
catch_threshold = .8
RT_threshold = 10000

## Data cleaning 
***Super important: df_clean is overwritten after each step of data cleaning***

**Reject subjects**
- Practice accuracy in conditional branching block
- Catch trial
- Environmental noise & audio device (may not need to use them as criteria)

**Reject trials**
- Task-relevant trials
- Extreme RT

### Reject subjects 

In [24]:
df_catch.groupby('participant_id')['response_correct'].count()

participant_id
0097603477ed55788b94a36df69f1bd7a071cdd1dc3df0f06c91edfc7401f39e    26
00d1c966a138b621ff1fa81d1b1e9e4370da03fea970cd89d3b62912191629ee    26
06c85661bb53c8683cdfedcb43aa14a04dce6c362d0cfe1c642961af74597273    26
07bd139a6d306ea8abb5d14f46259b99a4be49551319dd7bbf21ca871b565822    26
0f4126524237a59c94a37288e80a7da59ecda8437ded7902157a66313d9bb026    26
1d7b3e90bee82a78a33ef23a1c42e0a0edf98658382e82707137b15ba2763ce0    26
204ec71efa357ec23e14adb8c01199e786d0a28725714074cf1cb7602309ecbb    26
284211100c491c17fdf47a2751dc63a24bf884ebfdb6158d127c10f2109b7241    26
29176d5b94bbdcc7fc39850b8e42d221e4907e4b3aa6c76fc694ff59a21b4273    26
2c711231f47813aa8cd2120d86bec06052ef0ddd0351448721d1bc825be74b69    26
31aad6fa89d4b68139ace5e09b313f7cfa50e4bac9f55ea5dbd33f93d2df5fdb    26
348dc50420bbc29c1b5aeeea90af3e36cb7ef9688316cd34b4bc05743d7c55da    26
50bc7439a90a5aa6ac0b426ad2fc96da4497958fabee595a176da3b5cd08308d    26
51d74d50a0c7618caf43e2d4b1daf3ae9642783bdd79a063cb93b96b902323

In [25]:
## How many subjects miss the catch trial 
catch_trials = ['Catch_beat','Catch_bag']
catch_trials = ['Catch_trials'] ## for exp8abc

df_catch = df[(df['trial_template'].isin(catch_trials))].reset_index(drop = True) # reset index from 1
catch_acc = df_catch.groupby('sub_id')['response_correct'].sum()/df_catch.groupby('sub_id')['response_correct'].count()
catch_acc[catch_acc < catch_threshold]

sub_id
06c85    0.461538
759fe    0.692308
c9f49    0.576923
f1726    0.653846
Name: response_correct, dtype: float64

In [26]:
## How many subjects pass the practice > 0.75 on the easiest ontime trials 
df_clean = df[df['branch_failpass'] == 'pass'].reset_index(drop = True)
print('How many subjects passed the practice trials:', len(df_clean['sub_id'].unique())
      ,'out of',len(df['sub_id'].unique()))

How many subjects passed the practice trials: 28 out of 32


In [27]:
## How many subjects had a bad environmental noise and device
df_noise = df[(df['response_name'] == 'survey_noise')].reset_index(drop = True)
noise = df_noise.groupby('sub_id')['response_value'].sum()
df_device = df[(df['response_name'] == 'survey_headphone')].reset_index(drop = True)
df_device[['sub_id','response_value']]

,sub_id,response_value


### Task relevant trials

In [28]:
## subj id
subID = df['sub_id'].unique()

In [29]:
## All condition
conds = ['single_300ms_delay_2_51','single_300ms_delay_2_52','single_300ms_delay_2_53','single_300ms_delay_2_54','single_300ms_delay_2_55','single_300ms_delay_2_56',
        'single_300ms_delay_2_81','single_300ms_delay_2_82','single_300ms_delay_2_83','single_300ms_delay_2_84','single_300ms_delay_2_85','single_300ms_delay_2_86',
        'single_300ms_delay_2_91','single_300ms_delay_2_92','single_300ms_delay_2_93','single_300ms_delay_2_94','single_300ms_delay_2_95','single_300ms_delay_2_96',
        'single_300ms_delay_4_51','single_300ms_delay_4_52','single_300ms_delay_4_53','single_300ms_delay_4_54','single_300ms_delay_4_55','single_300ms_delay_4_56',
        'single_300ms_delay_4_81','single_300ms_delay_4_82','single_300ms_delay_4_83','single_300ms_delay_4_84','single_300ms_delay_4_85','single_300ms_delay_4_86',
        'single_300ms_delay_4_91','single_300ms_delay_4_92','single_300ms_delay_4_93','single_300ms_delay_4_94','single_300ms_delay_4_95','single_300ms_delay_4_96']

In [30]:
## Conditions short
conds = ['single_300ms_delay_2_51','single_300ms_delay_2_52','single_300ms_delay_2_53','single_300ms_delay_2_54','single_300ms_delay_2_55','single_300ms_delay_2_56',
        'single_300ms_delay_2_81','single_300ms_delay_2_82','single_300ms_delay_2_83','single_300ms_delay_2_84','single_300ms_delay_2_85','single_300ms_delay_2_86',
        'single_300ms_delay_2_91','single_300ms_delay_2_92','single_300ms_delay_2_93','single_300ms_delay_2_94','single_300ms_delay_2_95','single_300ms_delay_2_96']

In [31]:
## Conditions long
conds = ['single_300ms_delay_4_51','single_300ms_delay_4_52','single_300ms_delay_4_53','single_300ms_delay_4_54','single_300ms_delay_4_55','single_300ms_delay_4_56',
        'single_300ms_delay_4_81','single_300ms_delay_4_82','single_300ms_delay_4_83','single_300ms_delay_4_84','single_300ms_delay_4_85','single_300ms_delay_4_86',
        'single_300ms_delay_4_91','single_300ms_delay_4_92','single_300ms_delay_4_93','single_300ms_delay_4_94','single_300ms_delay_4_95','single_300ms_delay_4_96']

In [32]:
## Tasks & Conditions pretest
pretest_conds = ['short_tone_1','long_tone_6']
subID = df_clean['sub_id'].unique()

In [33]:
## select the main trials 
df_clean = df[(df['stimuli_presented'].isin(conds)) & (df['task'] == "maintaskTrials")].reset_index(drop = True)

In [34]:
## select the pretest trials 
df_clean = df[(df['stimuli_presented'].isin(conds)) & (df['task'] == "pretestTrials")].reset_index(drop = True)

In [35]:
## Long format for each condition
# B/T: presenting order & response key
# W/I: Onset time, comparison length, delay length
df_clean['onset'] = df_clean['stimuli_presented'].apply(lambda x: x.split("_")[-1][0])
df_clean['delay'] = df_clean['stimuli_presented'].apply(lambda x: x.split("_")[-2])
df_clean['comparison'] = df_clean['stimuli_presented'].apply(lambda x: x.split("_")[-1][1])
df_clean['order'] = df_clean['group_id'].apply(lambda x: x.split("_")[-1])
df_clean['key'] = df_clean['group_id'].apply(lambda x: x.split("_")[-2])

### Extreme reaction time

In [36]:
df_clean_longRT = df_clean[df_clean['response_rt'] > RT_threshold]

## Analyze noise and overall accuracy/catch accuracy

In [37]:
noise = noise.astype(int) # turn str to int to do correlaton
acc = df_clean.groupby('sub_id')['response_correct'].sum()/\
df_clean.groupby('sub_id')['response_correct'].count()

In [38]:
corr = np.corrcoef(noise,acc)[0,1]
print(corr, '\n')
corr = np.corrcoef(noise,catch_acc)[0,1]
print(corr, '\n')

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 32 and the array at index 1 has size 0

In [ ]:
# plot the data...
plt.scatter(acc,noise,color='r')
plt.xlabel('Accuracy')
plt.ylabel('Noise')
plt.show()

In [ ]:
# plot the data...
plt.scatter(catch_acc,noise,color='r')
plt.hist(catch_acc,noise)
plt.xlabel('Accuracy')
plt.ylabel('Noise')
plt.show()

## Analyze pretest trials

In [39]:
df_pretest = df_clean[(df_clean['trial_template'].isin(pretest_trials))]
#print(df_pretest.groupby(['participant_id','stimuli_presented']).mean()['Correct'])
print(df_pretest.groupby(['stimuli_presented']).mean()['Correct'])
print(df_pretest.groupby(['stimuli_presented']).mean()['Shorter'])
print(np.mean(df_pretest['Shorter']))

NameError: name 'pretest_trials' is not defined

## Save the onset and comparison length to the csv file for R analysis

In [11]:
## Number of subjects, trials & conditions for each subjects
n_subj = len(df_clean['sub_id'].unique())
n_trial = len(df_clean)//15
n_conds = len(df_clean['stimuli_presented'].unique())
print('Participant_number:', n_subj,'Trial number:', n_trial,'Condition number:', n_conds, sep='\n')

Participant_number:
67
Trial number:
1929
Condition number:
36


In [40]:
## Group_by
overall_acc = df_clean.groupby(['sub_id']).mean()['Correct'].reset_index(drop = True)
overall_pps = df_clean.groupby(['sub_id']).mean()['Shorter'].reset_index(drop = True)
all_conds_acc = df_clean.groupby(['sub_id','onset','delay','comparison','order','key']).mean()['Correct'].reset_index(drop = True)
all_conds_pps = df_clean.groupby(['sub_id','onset','delay','comparison','order','key']).mean()['Shorter'].reset_index(drop = True)

In [41]:
## save df to csv
df_clean.to_csv(r'/Users/t.z.cheng/Google_Drive/Research/Delaydoesmatter/real_exp/exp4_20CR12/results_shortlongdelay_2021/v2_20CR12_clean_n67_cleaned.csv', header=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/t.z.cheng/Google_Drive/Research/Delaydoesmatter/real_exp/exp4_20CR12/results_shortlongdelay_2021/v2_20CR12_clean_n67_cleaned.csv'

## Analyze trials of the main task

### Accuracy

In [42]:
Accuracy = []
Acc_cond = []
Easiest = []
Easiest_idx = [conds.index('single_300ms_delay_2_81'),conds.index('single_300ms_delay_2_86')]
# Easiest_idx = [conds.index('single_300ms_delay_4_81'),conds.index('single_300ms_delay_4_86')]
for sub in subID:
    tmpdf = df_clean[df_clean['participant_id'] == sub]
    for cond in conds:
        tmpAcc = np.mean(tmpdf[tmpdf['stimuli_presented'] == cond]['response_correct'])
        Acc_cond.append(tmpAcc)
    Easiest.append((Acc_cond[Easiest_idx[0]] + Acc_cond[Easiest_idx[-1]])/len(Easiest_idx)) # average accuracy of 81 and 86 
    Accuracy.append(Acc_cond)
    Acc_cond = []

ValueError: 'single_300ms_delay_2_81' is not in list

In [ ]:
## Outliers based on the accuracy of the easiest trials 
Acc_easiest = pd.Series(Easiest, index=df_clean['participant_id'].unique())
print(Acc_easiest.describe())
t = Acc_easiest[Acc_easiest >= threshold]
print("Subjects who have >=",threshold, "accuracy for the easiest trials:",len(t))

In [ ]:
df_Accuracy = pd.DataFrame(data = Accuracy, index = subID, columns = conds)
Acc_final_sample = df_Accuracy[Acc_easiest >= threshold]
Acc_final_sample.head()
print('Final sample size:', len(Acc_final_sample))
# Acc_final_sample.describe()

In [ ]:
df_Accuracy.index.name = 'participant_id'

In [ ]:
outliers = Acc_easiest < threshold
outliers = outliers[outliers == True]
df_clean_nooutlier = df_clean[~df_clean['participant_id'].isin(outliers.index)]
len(df_clean_nooutlier)/216

In [ ]:
len(Acc_final_sample)

In [ ]:
## Save the dataset to csv
# df_clean_nooutlier.to_csv(path_or_buf = '/Users/t.z.cheng/Google_Drive/Research/Delaydoesmatter/real_exp/exp4_20CR12/results/20CR12_clean_n53.csv', index=True)

In [ ]:
## Early, ontime, late short delay
early = [conds.index('single_300ms_delay_2_51'),conds.index('single_300ms_delay_2_52'),conds.index('single_300ms_delay_2_53'),conds.index('single_300ms_delay_2_54'),conds.index('single_300ms_delay_2_55'),conds.index('single_300ms_delay_2_56')]
ontime = [conds.index('single_300ms_delay_2_81'),conds.index('single_300ms_delay_2_82'),conds.index('single_300ms_delay_2_83'),conds.index('single_300ms_delay_2_84'),conds.index('single_300ms_delay_2_85'),conds.index('single_300ms_delay_2_86')]
late = [conds.index('single_300ms_delay_2_91'),conds.index('single_300ms_delay_2_92'),conds.index('single_300ms_delay_2_93'),conds.index('single_300ms_delay_2_94'),conds.index('single_300ms_delay_2_95'),conds.index('single_300ms_delay_2_96')]
Acc_early = Acc_final_sample.iloc[:, early].mean(axis = 1)
Acc_ontime = Acc_final_sample.iloc[:, ontime].mean(axis = 1)
Acc_late = Acc_final_sample.iloc[:, late].mean(axis = 1)
print("Accuracy")
print("Early:", np.mean(Acc_early))
print("Ontime:", np.mean(Acc_ontime))
print("Late:",np.mean(Acc_late))

In [ ]:
## Early, ontime, late long delay
early = [conds.index('single_300ms_delay_4_51'),conds.index('single_300ms_delay_4_52'),conds.index('single_300ms_delay_4_53'),conds.index('single_300ms_delay_4_54'),conds.index('single_300ms_delay_4_55'),conds.index('single_300ms_delay_4_56')]
ontime = [conds.index('single_300ms_delay_4_81'),conds.index('single_300ms_delay_4_82'),conds.index('single_300ms_delay_4_83'),conds.index('single_300ms_delay_4_84'),conds.index('single_300ms_delay_4_85'),conds.index('single_300ms_delay_4_86')]
late = [conds.index('single_300ms_delay_4_91'),conds.index('single_300ms_delay_4_92'),conds.index('single_300ms_delay_4_93'),conds.index('single_300ms_delay_4_94'),conds.index('single_300ms_delay_4_95'),conds.index('single_300ms_delay_4_96')]
Acc_early = Acc_final_sample.iloc[:, early].mean(axis = 1)
Acc_ontime = Acc_final_sample.iloc[:, ontime].mean(axis = 1)
Acc_late = Acc_final_sample.iloc[:, late].mean(axis = 1)
print("Accuracy")
print("Early:", np.mean(Acc_early))
print("Ontime:", np.mean(Acc_ontime))
print("Late:",np.mean(Acc_late))

In [ ]:
## Visualization: bar plot
ACC_plot = pd.DataFrame({'Conditions': conds, 'acc': np.mean(Acc_final_sample, axis = 0)})
ax = ACC_plot.plot.bar(rot = 0)

### Proportion short

In [43]:
resp_short = []
resp_short = [True if d == 'Shorter' else False for d in df_clean['response_value']]

In [44]:
df_clean['resp_short'] = resp_short

In [45]:
PPS = []
PPS_cond = []
for sub in subID:
    tmpdf = df_clean[df_clean['participant_id'] == sub]
    for cond in conds:
        tmpPPS = np.mean(tmpdf[tmpdf['stimuli_presented'] == cond]['resp_short'])
        PPS_cond.append(tmpPPS)
    PPS.append(PPS_cond)
    PPS_cond = []

In [46]:
df_PPS = pd.DataFrame(data = PPS, index = subID, columns = conds)

In [47]:
PPS_final_sample = df_PPS[Acc_easiest >= threshold]
PPS_final_sample.head()
PPS_final_sample.describe()

NameError: name 'Acc_easiest' is not defined

In [48]:
## Early, ontime, late short
early = [conds.index('single_300ms_delay_2_51'),conds.index('single_300ms_delay_2_52'),conds.index('single_300ms_delay_2_53'),conds.index('single_300ms_delay_2_54'),conds.index('single_300ms_delay_2_55'),conds.index('single_300ms_delay_2_56')]
ontime = [conds.index('single_300ms_delay_2_81'),conds.index('single_300ms_delay_2_82'),conds.index('single_300ms_delay_2_83'),conds.index('single_300ms_delay_2_84'),conds.index('single_300ms_delay_2_85'),conds.index('single_300ms_delay_2_86')]
late = [conds.index('single_300ms_delay_2_91'),conds.index('single_300ms_delay_2_92'),conds.index('single_300ms_delay_2_93'),conds.index('single_300ms_delay_2_94'),conds.index('single_300ms_delay_2_95'),conds.index('single_300ms_delay_2_96')]
PPS_early = PPS_final_sample.iloc[:, early].mean(axis = 1)
PPS_ontime = PPS_final_sample.iloc[:, ontime].mean(axis = 1)
PPS_late = PPS_final_sample.iloc[:, late].mean(axis = 1)
print("Proportion short")
print("Early:", np.mean(PPS_early))
print("Ontime:", np.mean(PPS_ontime))
print("Late:",np.mean(PPS_late))

ValueError: 'single_300ms_delay_2_51' is not in list

In [49]:
## Early, ontime, late long
early = [conds.index('single_300ms_delay_4_51'),conds.index('single_300ms_delay_4_52'),conds.index('single_300ms_delay_4_53'),conds.index('single_300ms_delay_4_54'),conds.index('single_300ms_delay_4_55'),conds.index('single_300ms_delay_4_56')]
ontime = [conds.index('single_300ms_delay_4_81'),conds.index('single_300ms_delay_4_82'),conds.index('single_300ms_delay_4_83'),conds.index('single_300ms_delay_4_84'),conds.index('single_300ms_delay_4_85'),conds.index('single_300ms_delay_4_86')]
late = [conds.index('single_300ms_delay_4_91'),conds.index('single_300ms_delay_4_92'),conds.index('single_300ms_delay_4_93'),conds.index('single_300ms_delay_4_94'),conds.index('single_300ms_delay_4_95'),conds.index('single_300ms_delay_4_96')]
PPS_early = PPS_final_sample.iloc[:, early].mean(axis = 1)
PPS_ontime = PPS_final_sample.iloc[:, ontime].mean(axis = 1)
PPS_late = PPS_final_sample.iloc[:, late].mean(axis = 1)
print("Proportion short")
print("Early:", np.mean(PPS_early))
print("Ontime:", np.mean(PPS_ontime))
print("Late:",np.mean(PPS_late))

NameError: name 'PPS_final_sample' is not defined

In [ ]:
## Visualization: bar plot
PSS_plot = pd.DataFrame({'Conditions': conds, 'PSS': np.mean(PPS_final_sample, axis = 0)})
ax = PSS_plot.plot.bar(rot = 0)

### Statistic tests

In [ ]:
print(stats.ttest_rel(PPS_early, PPS_late))
print(stats.ttest_rel(PPS_early, PPS_ontime))
print(stats.ttest_rel(PPS_late, PPS_ontime))